In [ ]:
from IPython import display
import math
from pprint import pprint
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid', context='talk', palette='Dark2')

In [ ]:
f = open(r"gme_titles.txt","r")


In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

sia = SIA()
results = []

for line in f:
    line = line.strip('\n')
    pol_score = sia.polarity_scores(line)
    pol_score['headline'] = line
    results.append(pol_score)





In [ ]:
print(results[:3])

In [ ]:
df = pd.DataFrame.from_records(results)
df.head()

In [3]:
import dbconn
import reddit_scraper
import pandas as pd

df = pd.DataFrame(reddit_scraper.search_pushshift_titles('GME',100,0),columns = ['Title','Context','Timestamp'])
df2 = pd.DataFrame(reddit_scraper.search_pushshift_titles('TSLA',100,0),columns = ['Title','Context','Timestamp'])
df3 = pd.DataFrame(reddit_scraper.search_pushshift_titles('AMC',100,0),columns = ['Title','Context','Timestamp'])
df4 = pd.DataFrame(reddit_scraper.search_pushshift_titles('CLOV',100,0),columns = ['Title','Context','Timestamp'])
df5 = pd.DataFrame(reddit_scraper.search_pushshift_titles('PLTR',100,0),columns = ['Title','Context','Timestamp'])
df6 = pd.DataFrame(reddit_scraper.search_pushshift_titles('AAPL',100,0),columns = ['Title','Context','Timestamp'])
df7 = pd.DataFrame(reddit_scraper.search_pushshift_titles('BB',100,0),columns = ['Title','Context','Timestamp'])
df8 = pd.DataFrame(reddit_scraper.search_pushshift_titles('AMD',100,0),columns = ['Title','Context','Timestamp'])
df9 = pd.DataFrame(reddit_scraper.search_pushshift_titles('SNDL',100,0),columns = ['Title','Context','Timestamp'])
df10 = pd.DataFrame(reddit_scraper.search_pushshift_titles('PTON',100,0),columns = ['Title','Context','Timestamp'])


100
100
100
100
100
100
100
100
100
100


In [11]:
# df.insert(0,'Ticker','GME')
df2.insert(0,'Ticker','TSLA')
df3.insert(0,'Ticker','AMC')
df4.insert(0,'Ticker','CLOV')
df5.insert(0,'Ticker','PLTR')
df6.insert(0,'Ticker','AAPL')
df7.insert(0,'Ticker','BB')
df8.insert(0,'Ticker','AMD')
df9.insert(0,'Ticker','SNDL')
df10.insert(0,'Ticker','PTON')

In [12]:
df_total = pd.concat([df,df2,df3,df4,df5,df6,df7,df8,df9,df10],ignore_index=True)

In [8]:
import numpy as np

test = np.array(reddit_scraper.search_pushshift_titles('GME',2,0))
print(test.shape)

(100, 3)


In [13]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Ticker     1000 non-null   object
 1   Title      1000 non-null   object
 2   Context    1000 non-null   object
 3   Timestamp  1000 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 31.4+ KB


In [14]:
df_total2 = df_total.replace('[^a-zA-Z ]', ' ', regex=True)

In [16]:
df_total2.tail()

,Ticker,Title,Context,Timestamp
995,PTON,PTON,On its way to a new high or simply overvalued...,1605981816
996,PTON,PTON BY FRIDAY,or ban,1605910985
997,PTON,PLTR SOLO or PTON,I have left Where to yolo Will they kee...,1605904829
998,PTON,Zm or PTON bought at high what would you do s...,Holding Zm PTON Made a Bad bad move a...,1605844040
999,PTON,PTON has been going down for the past month or...,PTON has been going down for the past month or...,1605713602


In [20]:
print(df_total2.groupby("Ticker").count())

        Title  Context  Timestamp
Ticker                           
AAPL      100      100        100
AMC       100      100        100
AMD       100      100        100
BB        100      100        100
CLOV      100      100        100
GME       100      100        100
PLTR      100      100        100
PTON      100      100        100
SNDL      100      100        100
TSLA      100      100        100


In [23]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [56]:
def lemmatize_pipe(doc):
    lemma_list = [str(tok.lemma_).lower() for tok in doc
                  if tok.is_alpha] 
    return lemma_list

def preprocess_pipe(texts):
    preproc_pipe = []
    for doc in nlp.pipe(texts, batch_size=20):
        preproc_pipe.append(lemmatize_pipe(doc))
    return preproc_pipe

def lemmatize(text):
    doc = nlp(text)
    lemma_list = [str(tok.lemma_) for tok in doc
                  if tok.is_alpha]
    lemma_context = " ".join(lemma_list)
    return lemma_context

In [57]:
df_total2['Clean Context'] = df_total2['Context'].apply(lemmatize)

In [58]:
df_total2.head()

,Ticker,Title,Context,Timestamp,Clean Context,Compound
0,GME,Can autozone and gme compare,Autozone is just one of many companies that se...,1618868839,autozone be just one of many company that sell...,0.4019
1,GME,Took out a Balance Transfer from Discover for ...,Adding to my X XXX shares I already HODL,1618868772,add to my x xxx share i already hodl,0.2960
2,GME,AMC GME F U R Y Level I,Level II added some power electronics for...,1618868567,level ii add some power electronic for add rag...,-0.5574
3,GME,I m not buying another share of GME,Until Thursday when I get paid Still going...,1618868405,until thursday when i get pay still go to be a...,0.6858
4,GME,Idea for a GME meme video Hercules Hydra battle,I feel like this OG fight scene is perfect for...,1618868252,i feel like this og fight scene be perfect for...,0.7183


In [51]:
df_total2['Clean Context'] = " ".join(df_total2['Context'])

In [52]:
df_total2

,Ticker,Title,Context,Timestamp,Clean Context,Compound
0,GME,Can autozone and gme compare,Autozone is just one of many companies that se...,1618868839,Autozone is just one of many companies that se...,0.4019
1,GME,Took out a Balance Transfer from Discover for ...,Adding to my X XXX shares I already HODL,1618868772,Autozone is just one of many companies that se...,0.2960
2,GME,AMC GME F U R Y Level I,Level II added some power electronics for...,1618868567,Autozone is just one of many companies that se...,-0.5574
3,GME,I m not buying another share of GME,Until Thursday when I get paid Still going...,1618868405,Autozone is just one of many companies that se...,0.6858
4,GME,Idea for a GME meme video Hercules Hydra battle,I feel like this OG fight scene is perfect for...,1618868252,Autozone is just one of many companies that se...,0.7183
...,...,...,...,...,...,...
995,PTON,PTON,On its way to a new high or simply overvalued...,1605981816,Autozone is just one of many companies that se...,0.0000
996,PTON,PTON BY FRIDAY,or ban,1605910985,Autozone is just one of many companies that se...,-0.5574
997,PTON,PLTR SOLO or PTON,I have left Where to yolo Will they kee...,1605904829,Autozone is just one of many companies that se...,0.2732
998,PTON,Zm or PTON bought at high what would you do s...,Holding Zm PTON Made a Bad bad move a...,1605844040,Autozone is just one of many companies that se...,0.3737


In [59]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
sia = SIA()

df_total2['Compound'] = [sia.polarity_scores(x)['compound'] for x in df_total2['Context']]
df_total2['Clean Compound'] = [sia.polarity_scores(x)['compound'] for x in df_total2['Clean Context']]


In [60]:
df_total2.head()

,Ticker,Title,Context,Timestamp,Clean Context,Compound,Clean Compound
0,GME,Can autozone and gme compare,Autozone is just one of many companies that se...,1618868839,autozone be just one of many company that sell...,0.4019,0.4019
1,GME,Took out a Balance Transfer from Discover for ...,Adding to my X XXX shares I already HODL,1618868772,add to my x xxx share i already hodl,0.2960,0.2960
2,GME,AMC GME F U R Y Level I,Level II added some power electronics for...,1618868567,level ii add some power electronic for add rag...,-0.5574,-0.5574
3,GME,I m not buying another share of GME,Until Thursday when I get paid Still going...,1618868405,until thursday when i get pay still go to be a...,0.6858,0.7351
4,GME,Idea for a GME meme video Hercules Hydra battle,I feel like this OG fight scene is perfect for...,1618868252,i feel like this og fight scene be perfect for...,0.7183,-0.6956
